In [3]:
! pip install mlxtend

In [9]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


import pandas as pd
import numpy as np

In [89]:
df = pd.read_csv(r'/content/drive/MyDrive/MBA/Sample_Data_For_MBA_2021.csv')

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331652 entries, 0 to 331651
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   c_email       331652 non-null  object
 1   category_rr3  331652 non-null  object
dtypes: object(2)
memory usage: 5.1+ MB


In [91]:
df.head(10)

,c_email,category_rr3
0,13alfik@gmail.com,Adaptor / Konektor
1,a.nurmandinata@gmail.com,Adaptor / Konektor
2,a.singarimbun.as@gmail.com,Adaptor / Konektor
3,abdulra.alj@gmail.com,Adaptor / Konektor
4,abhey253@gmail.com,Adaptor / Konektor
5,abraham@panjaitan.me,Adaptor / Konektor
6,aceaeon439@gmail.com,Adaptor / Konektor
7,acelippojember@gmail.com,Adaptor / Konektor
8,achmadfadillah794@gmail.com,Adaptor / Konektor
9,adelinejess@yahoo.com,Adaptor / Konektor


# Build Transactional Data

In [92]:
df['flag'] = 1
trx = df.groupby(['c_email','category_rr3'])['flag'].count().reset_index()
trx

,c_email,category_rr3,flag
0,'-,Bohlam Lampu,1
1,'-,Boneka Kain,1
2,'-,Cairan Pembersih,1
3,'-,Cermin,1
4,'-,Hanger Baju,1
...,...,...,...
331647,zzziatoo@gmail.com,Payung Taman dan Gazebo,1
331648,zzziatoo@gmail.com,Pelatih Ketangkasan,1
331649,zzziatoo@gmail.com,Perlengkapan Fitness,1
331650,zzziatoo@gmail.com,Sekering,1


In [93]:
trx = trx[~(trx['c_email']=="'-")]
trx.shape

(331633, 3)

In [94]:
trx = (trx.groupby(['c_email', 'category_rr3'])['flag'].sum().unstack().reset_index().fillna(0).set_index('c_email')).astype('int64')
trx

category_rr3,Adaptor / Konektor,Air Conditioner,Aki dan Aksesoris,Aksesoris Alat Listrik,Aksesoris Audio dan Video,Aksesoris Bangunan,Aksesoris Cuci Baju,Aksesoris Eksterior Mobil,Aksesoris Fashion,Aksesoris Fotografi,Aksesoris Hewan Peliharaan,Aksesoris Interior Mobil,Aksesoris Kamar Mandi,Aksesoris Kebersihan,Aksesoris Lampu,Aksesoris Meja Makan,Aksesoris Motor,Aksesoris Olahraga Air,Aksesoris Pengencang,Aksesoris Pengendara Motor,Aksesoris Peralatan Elektronik,Aksesoris Pickup,Aksesoris Sepatu,Aksesoris Setir,Aksesoris TV,Aksesoris Taman,Aksesoris Telepon Genggam,Aksesoris Unit Instalasi,Aksesoris Wisata,Aksesoris dan Tanaman Artifisial,Akuarium dan Aksesoris,Alarm dan Keamanan Rumah,Alat Derek,Alat Las dan Aksesoris,Alat Pemanggang,Alat Pembersih Lantai,Alat Pembersih Uap,Alat Penanda,Alat Penanda Dan Pengukur,Alat Pengangkut,...,Stoples dan Wadah Makanan,Suplemen,Tali,Tanaman dan Rumput,Tang,Tangga Khusus,Tangga Lipat,Tangga Portable,Tape,Tas,Tas ASI dan perlengkapan Ibu Hamil,Tas Fashion,Tas Olahraga,Tas Perkakas,Teko dan Alat Masak Kaca,Tekstil Natal,Telepon dan Fax,Televisi,Tempat Bekal Makanan,Tempat Kosmetik,Tempat Sampah Dalam Ruangan,Tempat Sampah Luar Ruangan,Tempat Tidur Bayi dan Aksesoris,Temporary,Tenda dan Kantong Tidur,Termos,Teropong dan Lensa Pembidik,Timbangan,Timbangan Dapur dan Timer,Toilet,Troli Dapur,Troli Hidrolik,Troli Salon,Turnbuckles,"Vise, Ragum dan Kelm",Voucher Belanja,Voucher Jasa dan Hiburan,Voucher Makanan dan Minuman,Wajan dan Kuali,Wiper dan Bumper
c_email,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001bos@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00putri.amalia@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
01.abid.am@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0126dsw@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0163358150@yahoo.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzy.store01@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
zzz.fatah@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
zzzainalarifin@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
frequent = apriori(trx,min_support=0.002,use_colnames=True)
frequent

,support,itemsets
0,0.010537,(Adaptor / Konektor)
1,0.013412,(Air Conditioner)
2,0.002527,(Aksesoris Audio dan Video)
3,0.005083,(Aksesoris Cuci Baju)
4,0.012419,(Aksesoris Hewan Peliharaan)
...,...,...
803,0.002038,"(Sikat dan Spons, Pel dan Aksesoris, Bohlam La..."
804,0.002438,"(Sikat dan Spons, Pel dan Aksesoris, Cairan Pe..."
805,0.002267,"(Sikat dan Spons, Pel dan Aksesoris, Keset)"
806,0.002208,"(Seprai dan Pelindung Matras, Selimut dan Peli..."


In [114]:
frequent.sample(10)

,support,itemsets
183,0.002408,(Sampo Hewan Peliharaan)
141,0.021926,(Penghisap Debu)
169,0.002490,(Pisau Lipat Serbaguna)
515,0.003490,"(Pengharum Ruangan, Keset)"
227,0.002534,"(Aksesoris Hewan Peliharaan, Makanan Hewan)"
401,0.002275,"(Gembok dan Pengunci, Bohlam Lampu)"
466,0.002216,"(Selang dan Aksesoris, Cairan Pembersih)"
188,0.015775,(Selang dan Aksesoris)
174,0.051587,(Rak Besi)
200,0.021829,(Sikat dan Spons)


In [109]:
frequent.describe()

,support
count,808.000000
mean,0.004863
std,0.006235
min,0.002001
25%,0.002273
50%,0.002749
75%,0.004025
max,0.063524


In [141]:
rules = association_rules(frequent,metric='support',support_only=True,min_threshold=0.004)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Aksesoris Kamar Mandi),(Aksesoris Meja Makan),NaN,NaN,0.004668,NaN,NaN,NaN,NaN
1,(Aksesoris Meja Makan),(Aksesoris Kamar Mandi),NaN,NaN,0.004668,NaN,NaN,NaN,NaN
2,(Aksesoris Kamar Mandi),(Bohlam Lampu),NaN,NaN,0.005491,NaN,NaN,NaN,NaN
3,(Bohlam Lampu),(Aksesoris Kamar Mandi),NaN,NaN,0.005491,NaN,NaN,NaN,NaN
4,(Aksesoris Kamar Mandi),(Cairan Pembersih),NaN,NaN,0.004024,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
77,(Stoples dan Wadah Makanan),(Peralatan Minum),NaN,NaN,0.004461,NaN,NaN,NaN,NaN
78,(Sapu dan Pengki),(Sikat dan Spons),NaN,NaN,0.004350,NaN,NaN,NaN,NaN
79,(Sikat dan Spons),(Sapu dan Pengki),NaN,NaN,0.004350,NaN,NaN,NaN,NaN
80,(Seprai dan Pelindung Matras),(Selimut dan Pelindung Selimut),NaN,NaN,0.010463,NaN,NaN,NaN,NaN


In [146]:
output = rules[['antecedents','consequents','support']]
               

output['antecedents'].duplicated().sum()

58

In [147]:
cols = ['antecedents','consequents']
output[cols] = output[cols].applymap(lambda x: tuple(x))

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [148]:
output

,antecedents,consequents,support
0,"(Aksesoris Kamar Mandi,)","(Aksesoris Meja Makan,)",0.004668
1,"(Aksesoris Meja Makan,)","(Aksesoris Kamar Mandi,)",0.004668
2,"(Aksesoris Kamar Mandi,)","(Bohlam Lampu,)",0.005491
3,"(Bohlam Lampu,)","(Aksesoris Kamar Mandi,)",0.005491
4,"(Aksesoris Kamar Mandi,)","(Cairan Pembersih,)",0.004024
...,...,...,...
77,"(Stoples dan Wadah Makanan,)","(Peralatan Minum,)",0.004461
78,"(Sapu dan Pengki,)","(Sikat dan Spons,)",0.004350
79,"(Sikat dan Spons,)","(Sapu dan Pengki,)",0.004350
80,"(Seprai dan Pelindung Matras,)","(Selimut dan Pelindung Selimut,)",0.010463


In [149]:
output = output.explode('antecedents').reset_index(drop=True).explode('consequents').reset_index(drop=True)

In [151]:
output.sort_values('support',ascending=False)

,antecedents,consequents,support
35,Bohlam Lampu,Lampu Indoor,0.011204
34,Lampu Indoor,Bohlam Lampu,0.011204
81,Selimut dan Pelindung Selimut,Seprai dan Pelindung Matras,0.010463
80,Seprai dan Pelindung Matras,Selimut dan Pelindung Selimut,0.010463
70,Kursi Taman,Meja Taman,0.008010
...,...,...,...
4,Aksesoris Kamar Mandi,Cairan Pembersih,0.004024
51,Minuman,Camilan,0.004016
50,Camilan,Minuman,0.004016
28,Cairan Pembersih,Bohlam Lampu,0.004001


In [158]:
output[(output['antecedents']=='Meja Taman') | (output['consequents']=='Meja Taman')].sort_values('support',ascending=False)

,antecedents,consequents,support
70,Kursi Taman,Meja Taman,0.00801
71,Meja Taman,Kursi Taman,0.00801


In [161]:
output.drop_duplicates(subset='support',keep='first',inplace=True)

In [162]:
output

,antecedents,consequents,support
0,Aksesoris Kamar Mandi,Aksesoris Meja Makan,0.004668
2,Aksesoris Kamar Mandi,Bohlam Lampu,0.005491
4,Aksesoris Kamar Mandi,Cairan Pembersih,0.004024
6,Aksesoris Kamar Mandi,Keset,0.005602
8,Aksesoris Kamar Mandi,Kipas Angin dan Aksesoris,0.004713
10,Aksesoris Kamar Mandi,Pel dan Aksesoris,0.004913
12,Aksesoris Kamar Mandi,Pengharum Ruangan,0.004253
14,Sikat dan Spons,Aksesoris Kamar Mandi,0.004401
16,Tempat Sampah Dalam Ruangan,Aksesoris Kamar Mandi,0.004327
18,Aksesoris Meja Makan,Keset,0.004364
